# Packages

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import os
from numpy import genfromtxt
from scipy import stats
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import keras
from keras.layers import Dense, Flatten, Reshape,Dropout
from keras.layers import Conv2D, MaxPooling2D, LSTM
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import timeit #package for recording the model running time
import time
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, Conv3D, MaxPooling3D, Reshape, BatchNormalization, MaxPooling2D
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam
from keras.optimizers import SGD  
from keras import backend as K
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold,StratifiedKFold,ShuffleSplit,StratifiedShuffleSplit
# Packages & Setup
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import  preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,f1_score,accuracy_score
import timeit
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

# Functions

In [ ]:
def win_seg(data,windowsize,overlap):#function for overlap segmentation
    length=int((data.shape[0]*data.shape[1]-windowsize)/(windowsize*overlap)+1)
    newdata=np.empty((length,windowsize, data.shape[2],1))
    data_dim=data.shape[2]
    layers=data.shape[3]
    data=data.reshape(-1,data_dim,layers)
    for i in range(0,length) :
        start=int(i*windowsize*overlap)
        end=int(start+windowsize)
        newdata[i]=data[start:end]
    return newdata
def lab_vote(data,windowsize):
    y_data=data.reshape(-1,windowsize,1,1)
    y_data=win_seg(y_data,windowsize,0.5)
    y_data=y_data.reshape(y_data.shape[0],y_data.shape[1],y_data.shape[2])
    y_data=stats.mode(y_data,axis=1)
    y_data=y_data.mode
    y_data=y_data.reshape(-1,1)
    y_data=np.float64(keras.utils.to_categorical(y_data))
    return y_data
def lab_vote_cat(data,windowsize): # non one-hot coding
    y_data=data.reshape(-1,windowsize,1,1)
    y_data=win_seg(y_data,windowsize,0.5)
    y_data=y_data.reshape(y_data.shape[0],y_data.shape[1],y_data.shape[2])
    y_data=stats.mode(y_data,axis=1)
    y_data=y_data.mode
    y_data=y_data.reshape(-1,1)
    return y_data
def write_csv(data):
    a = np.asarray(data)
    a.tofile('check.csv',sep=',',format='%10.5f')
def average(lst): 
    a = np.array(lst)
    return np.mean(a)
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []
    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()
    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)
def yindex(y): #all contineous lables treated as one class
    y_index=[0]
    label=0
    for i in range(1,len(y)):
        if y[i]==y[i-1]:
            y_index.append(label)
        else:
            label+=1
            y_index.append(label)
    return y_index
def continue_split(x,y,y_index): #get the last 10% of every contineous movement
    skf = StratifiedKFold(n_splits=10,shuffle=False)
    counter=0
    for train_index, test_index in skf.split(x, y_index):
        if counter<9:
            counter+=1
            continue
    x_data_a, x_data_b=x[train_index],x[test_index]
    y_data_a, y_data_b=y[train_index],y[test_index]
    return x_data_a, x_data_b, y_data_a, y_data_b,train_index,test_index

def quasiEVA(lable_array,threshold,result):
    j=0
    c=list(np.unique(lable_array)) #total number of posture classes
    c=[int(i) for i in c]
    rows=len(lable_array) #total number of postures
    #count consequtive postures
    count=np.zeros(shape=(rows,2)) #buffer to store all cases of contineous postures
    count[0,1]=1
    for i in range(rows-1):
        if lable_array[i]==lable_array[i+1]:
            count[j,1]+=1
            count[j,0]=lable_array[i]
        else:
            j+=1
            count[j,0]=lable_array[i+1]
            count[j,1]=1
    count=count[count[:,1]>0] #get ride of empty rows
    #count holding time
    HT=np.zeros(shape=(len(count),1))
    HT[:,0]=(count[:,1]+1)*0.5
    count=np.append(count,HT,1)
    for ind,val in enumerate(c):
            count_red=count[count[:,0]==val] #Get all the contineous occurance of posture i
            count_red_result=count_red[count_red[:,2]>threshold[ind]] # filtering out all the MHT above threshold
            result.iloc[[ind],[1]]=len(count_red_result[:,1]) #total count of breaching MHT
            result.iloc[[ind],[2]]=sum(count_red_result[:,2]) #total duration of breaching MHT
            result.iloc[[ind],[3]]=(len(count_red[:,1])/sum(count[:,2]))*60 #frequencies of activity i occurance in 1 minute
            result.iloc[[ind],[4]]=sum(count_red[:,1])/sum(count[:,1]) #proportion of activity i
            result.iloc[[ind],[5]]=max(count_red[:,2]) #max time of holding
    return result, count,count_red

# Apply both DNN and ML Models for Posture Recognition

## ML Model

In [ ]:
#loading the training and testing data, using R3 as example
buffer = np.float64(preprocessing.scale(genfromtxt('S3_X.csv', delimiter=',')))
x_data=buffer.reshape(-1,40,30,1)
x_data=win_seg(x_data,40,0.5) # data segmentation with 0.5 overlap
#majority vote on training label
buffer = np.float64(genfromtxt('S3_Y.csv', delimiter=','))-1 #0 based index
y_data=lab_vote(buffer,40)
#fix the dimension mismatch
if y_data.shape[1]<7:
    zeros=np.zeros((y_data.shape[0],7-y_data.shape[1]))
    y_data=np.hstack((y_data,zeros))
y_data2=lab_vote_cat(buffer,40) # for stratification purposes
y_index=yindex(y_data2)
x_data_a, x_data_b, y_data_a, y_data_b,train_index,test_index=continue_split(x_data,y_data,y_index)

# Load the ML full feature and construct the one with selected generalized features
# Export the best features
sel_features=pd.DataFrame()
fullfeatures=pd.read_csv("S3_Full Features.csv")
names=list(fullfeatures.columns.values)[1:]
sel_name_list=pd.read_csv("Generalized ML_SelectetedFeaturesNames.csv")
sel_name_list=list(sel_name_list["feature names"])
for index, val in enumerate(sel_name_list):
    if val:
        sel_features=pd.concat([sel_features,fullfeatures.iloc[:,index+1]],axis=1)

sel_features.to_csv("S3_Selecteted Features.csv")
# Data Prepration
X_ML_train, X_ML_test, X_ML_data,y_ML_train, y_ML_test, y_ML_data=preparation("S3_Selecteted Features.csv")
X_ML_train=X_ML_data[train_index]
y_ML_train=y_ML_data[train_index]
X_ML_test=X_ML_data[test_index]
y_ML_test=y_ML_data[test_index]
# Get the ML prediction Result
svm=SVC(gamma='auto',random_state=42)
svm.fit(X_ML_train,y_ML_train)
svm_pre=pd.DataFrame(data=svm.predict(X_ML_test))
acc=accuracy_score(y_ML_test, svm_pre)
f1=f1_score(y_ML_test, svm_pre,average='macro')
ML_Result,ML_Pre=TrainModels(X_ML_train, X_ML_test, y_ML_train, y_ML_test)
ML_Result.to_csv("S3_ML_Performance.csv")
ML_Pre.to_csv("S3_ML_SVM_Recognition.csv")

## DNN Model 

In [ ]:
#incremental training on first half
#Model Evaluation Metrics
acc_score=list()
f_score=list()
eopch_time_record=list()
oper_time_record=list()
i=0

#Reload the trained model on rest subjects
newmodel=load_model("rest.h5", custom_objects={'f1': f1})
#split the train data into training (training the model) and validation (tuning hypeparameters) by 8:2
X_training, X_validation, y_training, y_validation = train_test_split(x_data_a, y_data_a, test_size=0.20)
#Setup Model Parameters
data_dim = x_data_a.shape[2] #y of figure
timesteps = x_data_a.shape[1] #x of figure
num_classes = y_data_a.shape[1]
batchsize=300
epcoh=300
checkpointer = ModelCheckpoint(filepath="S3.h5", monitor='val_f1',verbose=1, mode='max', save_best_only=True)
time_callback = TimeHistory() #record the model training time for each epoch
callbacks_list = [checkpointer,time_callback]
newmodel.compile(loss=keras.losses.categorical_crossentropy,
          optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy',f1])
train_history=newmodel.fit(X_training, y_training,
          batch_size=batchsize, epochs=epcoh,callbacks=callbacks_list,
          validation_data=(X_validation, y_validation))
CNN_LSTM_model=load_model("S3.h5", custom_objects={'f1': f1})

#check the model test result
y_pred=CNN_LSTM_model.predict(x_data_b)
y_pred = np.argmax(y_pred, axis=1)
Y_test=np.argmax(y_data_b, axis=1)
acc_score.append(accuracy_score(Y_test, y_pred)) # Evaluation of accuracy
f_score.append(f1_score(Y_test, y_pred,average='macro')) # Evaluation of F1 score
performance=pd.DataFrame(columns=['Acc_score','Macro_Fscore'])
performance['Acc_score']=acc_score
performance['Macro_Fscore']=f_score
performance.to_csv("LeaveOneOutOnS3.csv")
recognition_results=pd.DataFrame(columns=['Prediction','Actual'])
recognition_results['Prediction']=y_pred
recognition_results['Actual']=Y_test
recognition_results.to_csv('LeaveOneOutOnS3_Recognition.csv')

# Compare Ergonomics Assessment Results

In [ ]:
# Load Activitie Recognition Result from csv as numpy array, using postures recognized from incremental DNN model as example
act=genfromtxt('LeaveOneOutOnS3_Recognition.csv', delimiter=',',skip_header =1) # change the file name to get postures from Ground-Truth (G), Incremental Model (I),and Personalized Model
act=act[:,-1]#last column is actual
# Define Threshold for Activities
threshold= np.array([3,3,3,30]) # Safety Threshold in seconds
lab_names= np.array(["BT","KN","SQ","ST"])
columns=['Activities','Count of MHT','Total Duration of MHT','Frequency of Exposture in 1 Minute','Proportion of Activities','Maximum Holding Time']
index=range(len(lab_names))
result = pd.DataFrame(index=index,columns=columns)
result.iloc[:,0]=lab_names
result = result.fillna(0) # with 0s rather than NaNs
# Get the Quasi-Realtime Evaluation Result
eva_result,count,count_red=quasiEVA(act,threshold,result)
eva_result